In [ ]:
import math

%run Market.ipynb
%run Policy_Loader.ipynb
%run PolicyCons.ipynb
%run AccountRiskControl.ipynb


class SingleStockRiskControl:
    
    def __init__(self,account,symbol,context,policy_name):
        self.account = account
        self.symbol = symbol
        self.context = context
        
        self.short = context['short']
        self.mid = context['mid']
        self.policy_name = policy_name
        
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.market = Market(context)
        self.pCons = PolicyCons()
        self.accountRisk = AccountRiskControl(account,symbol,context,policy_name)
    
    
    def get_max_cost_from_account_level(self,cur_date):
        total_asset = self.account.get_honor_total_asset()
        
        if(self.policy_loader.enable_account_level_position()==True):
            pass
        else:
            return total_asset
    
    #单支股票最大付出成本
    def get_max_cost_for_one_stock(self,cur_date):
        max_cost_from_account = self.get_max_cost_from_account_level(cur_date)
        
        if(self.policy_loader.enable_stock_level_position()==True):
            per = self.policy_loader.get_stock_level_max_position_per()
            
            total_money_for_one_stock = max_cost_from_account*per/100.0
        else:
            total_money_for_one_stock = max_cost_from_account
        
        return total_money_for_one_stock
        
        
#   当前要购买多少百分比的仓位
    def get_to_buy_percent(self,cur_date):
        percent = 0
        cur_shares = self.account.get_shares(self.symbol)
        if(cur_shares<=0):
            percent = self.policy_loader.adding_position_incre_per_first_part()
        else:
            open_deals = self.account.get_open_buy_deal_size(self.symbol)
            
            if(open_deals==0):
                print('dirty open deals ' + str(self.account.get_active_buy_deals(self.symbol)))
            elif(open_deals==1):
                percent = self.policy_loader.adding_position_incre_per_second_part()
            elif(open_deals==2):
                percent = self.policy_loader.adding_position_incre_per_third_part()
            elif(open_deals==3):
                percent = self.policy_loader.adding_position_incre_per_fourth_part()
            elif(open_deals==4):
                if(self.debug==True):
                    print(self.symbol+' '+cur_date+' already full space')
            else:
                print('dirty open deals ' + str(self.account.get_active_buy_deals(self.symbol)))
                
        return percent
        
        
#   当前最大可买多少股
    def get_shares_by_space(self, cur_date, cur_price):
        max_cost = self.get_max_cost_for_one_stock(cur_date)
        
        if(max_cost<=0):
            print('Warning dirty data max_cost for one stock '+cur_date+' '+str(max_cost))
            return 0
       
        to_buy_percent = self.get_to_buy_percent(cur_date)
        to_buy_stock_money = max_cost*to_buy_percent/100.0
        
        if(self.debug==True):
            print('max_cost:'+str(max_cost)+' to_buy_percent:'+str(to_buy_percent)+' to_buy_stock_money:'+str(to_buy_stock_money))
        
        today_cash = self.account.get_all_cash()
        if(to_buy_stock_money>today_cash):
            to_buy_stock_money = today_cash

        shares = to_buy_stock_money/cur_price
        
        if(shares>100000):
            return 100000
        
        return int(shares/100)*100
        
        
    # 根据暴露的风险计算每次购买的股数
    def get_shares_by_expose_risk(self,cur_date,cur_price):
        expose_risk_per = self.policy_loader.adding_position_expose_risk_percent()
        
        total_asset = self.account.get_honor_total_asset()
        
        risk_money = total_asset*expose_risk_per/100.0
        
        if(self.policy_loader.get_stop_loss_approach()==self.pCons.stop_loss_approach_fixed_percent):
            fix_percent = self.policy_loader.get_stop_loss_fixed_percent()
            
            price_gap = abs((fix_percent*cur_price)/100.0)
            
            shares = risk_money/price_gap
            
            today_cash = self.account.get_all_cash()
            if(cur_price*shares>today_cash):
                shares = today_cash/cur_price
            
            if(self.debug==True):
                print('to buy shares:' + str(shares) +' total_asset:'+str(total_asset)+' price_gap:'+str(price_gap))
            
            shares =  int(shares/100)*100
            
            if(shares>100000):
                return 100000
        
            return shares
        
        
        
        
    